# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet'])

# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nadine.Puetzer1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Nadine.Puetzer1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('Clean_Disaster_Data_2', engine)  
#print(df.head())
X = df.message
print(X[0])
Y = df[df.columns[4:]]
#print(Y.iloc[0])
#Y = df[df.columns[4]]
print(Y.iloc[[0]])

Weather update - a cold front from Cuba that could pass over Haiti
   related  request  offer  aid_related  medical_help  medical_products  \
0        1        0      0            0             0                 0   

   search_and_rescue  security  military  child_alone  ...  aid_centers  \
0                  0         0         0            0  ...            0   

   other_infrastructure  weather_related  floods  storm  fire  earthquake  \
0                     0                0       0      0     0           0   

   cold  other_weather  direct_report  
0     0              0              0  

[1 rows x 36 columns]


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline =  Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train,y_train)

# predict on test data
y_pred = pipeline.predict(X_test)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
for col in range(0,len(Y.columns)):
    cr = classification_report(y_test.values[col], y_pred_cv[col])
    print('Pred. Category: ', Y.columns[col], ' Accuracy: ', cr['accurancy'], ' Precision: ', cr['macro avg']['precision'] ,' Recall: ', cr['macro avg']['recall'])
    # Print complete Report per column:
    # print(Y.columns[col]+'\n', classification_report(y_test.values[col], y_pred[col]))

related
               precision    recall  f1-score   support

           0       1.00      0.97      0.99        36
           1       0.00      0.00      0.00         0

    accuracy                           0.97        36
   macro avg       0.50      0.49      0.49        36
weighted avg       1.00      0.97      0.99        36

request
               precision    recall  f1-score   support

           0       0.97      1.00      0.99        35
           1       0.00      0.00      0.00         1

    accuracy                           0.97        36
   macro avg       0.49      0.50      0.49        36
weighted avg       0.95      0.97      0.96        36

offer
               precision    recall  f1-score   support

           0       0.94      1.00      0.97        31
           1       1.00      0.60      0.75         5

    accuracy                           0.94        36
   macro avg       0.97      0.80      0.86        36
weighted avg       0.95      0.94      0.94      

c:\Users\Nadine.Puetzer1\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Nadine.Puetzer1\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
'''
Default parameters for estimator MultiOutputClassifier(estimator=RandomForestClassifier
(bootstrap=True,
ccp_alpha=0.0,
class_weight=None,
criterion='gini',
max_depth=None,
max_features='auto',
max_leaf_nodes=None,
max_samples=None,
min_impurity_decrease=0.0,
min_impurity_split=None,
min_samples_leaf=1,
min_samples_split=2,
min_weight_fraction_leaf=0.0,
n_estimators=100,
n_jobs=None,
oob_score=False,
random_state=None,
verbose=0,
warm_start=False))'''


# Create the grid
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
#       'clf__estimator__max_depth': [10, 20, 30, 40, 50],
#       'estimator__max_depth': [None, 10, 20, 30, 40, 50],
#       'estimator__max_features': ['auto', 'sqrt', 'log2'],
#       'clf__estimator__min_samples_leaf': [1, 2, 4],
#       'clf__estimator__bootstrap': [True, False],
        'clf__estimator__min_samples_split': [2, 3, 4]

    }    


# Apply GridSearchCV and print best parameters
cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, refit=True)
cv.fit(X_train, y_train)
print(cv.best_params_, cv.best_estimator_ )




### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_cv = cv.predict(X_test)

for col in range(0,len(Y.columns)):
    cr = classification_report(y_test.values[col], y_pred_cv[col])
    print('Pred. Category: ', Y.columns[col], ' Accuracy: ', cr['accurancy'], ' Precision: ', cr['macro avg']['precision'] ,' Recall: ', cr['macro avg']['recall'])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.